In [ ]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
import plotly.express as px
output_notebook()
from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt


from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as stats
import math

In [ ]:
import pprint
from RootInteractive.InteractiveDrawing.bokeh.bokehInteractiveParameters import *
pp = pprint.PrettyPrinter(width=220, compact=True)
#from RootInteractive.MLpipeline.NDFunctionInterface import  *
from RootInteractive.MLpipeline.MIForestErrPDF import *

# Random forest parameters 

In [ ]:
#%%time
n_estimators=200
n_jobs=100
npoints=1000000
max_depthBase=14
max_samples=0.1
regressorBase = RandomForestRegressor(n_estimators =n_estimators,n_jobs=n_jobs,max_depth=max_depthBase,max_samples=max_samples)
regressorLocal = RandomForestRegressor(n_estimators =n_estimators,n_jobs=n_jobs,max_samples=max_samples)

In [ ]:
arrayCompressionRelative10=[(".*",[("relative",10), ("code",0), ("zip",0), ("base64",0)])]

In [ ]:
# defined by user
calPath="/home/berki/Software/TPCQCVis/TPCQCVis/macro/TreeDump_run_523556.root"
fIn =ROOT.TFile.Open(calPath)
tree=fIn.Get("calibTree")

In [ ]:
tree.Show(0)
#tree.GetListOfAliases().Print()

## Import variables and aliases from the tree to panda

In [ ]:
#%%time
varList=[
    "roc","ly","lx","gy","gx","row","pad","padArea", ## position
    "isEdgePad","partition",           ## 
    "traceLength",                     ## trace length
    "GainMap",                         ## krypton gain map
    "Pedestals","Noise",               ## 
    "T0","Qtot",                       ## pulser properties          
    "N_Digits","Q_Max_Digits",         ## digits occumancy and Qmax 
    "N_Clusters","Q_Max","Q_Tot",      ## cluster Q-Max,Q_tot 
    #"fraction","expLambda",            ## ion tail prameters 
    "Sigma_Time","Sigma_Pad",          ## should be mean sigma of cluster finder in pad and time direction
    "A_Side","C_Side"
]

dfScan=tree2Panda(tree,varList,"roc>=0",columnMask=[["_fElements",""]])
print(dfScan.shape)

In [ ]:
dfScan["roc"]

In [ ]:
dfScan["dPedestals"]=dfScan["Pedestals"]-dfScan["Pedestals"].astype('int16')

# Fit base and local  propeties
* regressor with local X
* local filter regressor - mean,median,std filter

In [ ]:
#%%time
statDictionary={"mean":None,"median":None, "std":None}

varListN=["lx","traceLength","padArea"]
varListLocal=["lx","ly","roc"]
vars=[
    "Noise","N_Digits","N_Clusters",
    "Q_Max", "Q_Tot", "GainMap","Q_Max_Digits",
    "fraction","expLambda"
]
for var in vars:
    # base regression limitted deep
    regressorBase.fit(dfScan[varList],dfScan[var])
    dfScan[f"{var}RF0"]= regressorBase.predict(dfScan[varList])
    dfScan[f"{var}RF0_Ratio"]=dfScan[f"{var}"]/dfScan[f"{var}RF0"]
    # local regression  
    regressorLocal.fit(dfScan[varListLocal],dfScan[var])
    statDictionaryOut=predictRFStatNew(regressorLocal,dfScan[varListLocal].astype('float32').to_numpy(),statDictionary,n_jobs)
    dfScan[f"{var}RFL"]= regressorLocal.predict(dfScan[varListLocal])
    dfScan[f"{var}RFL_Ratio"]=dfScan[f"{var}"]/dfScan[f"{var}RFL"]
    dfScan[f"{var}RFL_Med"]=statDictionaryOut["median"]
    dfScan[f"{var}RFL_Std"]=statDictionaryOut["std"]
    dfScan[f"{var}RFLMed_Ratio"]=dfScan[f"{var}"]/dfScan[f"{var}RFL_Med"]
    print(f"Fit {var}")



In [ ]:
print(dfScan["NoiseRF0"].mean(),dfScan["NoiseRF0_Ratio"].mean())


# Make parameters and widgets

In [ ]:
variables=["lx","dPedestals","Noise","NoiseRF0","NoiseRF0_Ratio","N_Digits","N_DigitsRF0","N_DigitsRF0_Ratio","N_Clusters","N_ClustersRF0","N_ClustersRF0_Ratio",
           "Q_Max","Q_MaxRF0_Ratio","Q_Tot","Q_TotRF0_Ratio",
           "GainMap","Q_Max_Digits",
          "Sigma_Time","Sigma_Pad"]


parameterArray = [  
    {"name": "varX", "value":"Noise", "options": variables},
    {"name": "varY", "value":"N_Digits", "options": variables },
    {"name": "nbinsX", "value":30, "range":[5, 100]},
    {"name": "nbinsY", "value":30, "range":[5, 100]},
    #
    {"name": "nbinsGX", "value":50, "range":[30, 250]},
    {"name": "nbinsGY", "value":50, "range":[30, 250]},
    {"name": "yAxisTransform", "value":"linear", "options":["linear","sqrt","log"]},
]
#
parameterArray.extend(figureParameters["legend"]['parameterArray'])   # add legend configuration as in the default 
parameterArray.extend(figureParameters["markers"]['parameterArray'])   # add legend configuration as in the default 
#
widgetParams=[
              #['spinnerRange', ['qMax'], {}],
              ['spinnerRange', ['roc'],{"name": "roc"}],
              ['spinnerRange', ['row'],{"name": "row"}],
              ["multiSelect",["partition"],{"name":"partition"}],
              ['multiSelect',["isEdgePad"],{"name":"isEdgePad"}],
              #
              ['range', ['lx'],{"name": "lx"}],
              ['range', ['ly'],{"name": "ly"}],
              ['range', ['gx'],{"name": "gx"}],
              ['range', ['gy'],{"name": "gy"}],
              #
              ['spinnerRange', ['Noise'],{"name": "Noise"}],
              ['spinnerRange', ['NoiseRF0_Ratio'],{"name": "NoiseRF0_Ratio"}],
              ['spinnerRange', ['N_DigitsRF0_Ratio'],{"name": "N_DigitsRF0_Ratio"}],
              ['spinnerRange', ['N_ClustersRF0_Ratio'],{"name": "N_ClustersRF0_Ratio"}],
              ['range', ['dPedestals'],{"name": "dPedestals"}],
    
              #
              ['select', ['varX'], {"name": "varX"}],
              ['select', ['varY'], {"name": "varY"}],
              ['spinner', ['nbinsY'], {"name": "nbinsY"}],
              ['spinner', ['nbinsX'], {"name": "nbinsX"}],
              ['spinner', ['nbinsGY'], {"name": "nbinsGY"}],
              ['spinner', ['nbinsGX'], {"name": "nbinsGX"}],
              ['select', ['yAxisTransform'], {"name": "yAxisTransform"}],
    
]
#
widgetParams.extend(figureParameters["legend"]["widgets"])
widgetParams.extend(figureParameters["markers"]["widgets"])
#
widgetLayoutKine=[["roc","row","partition","isEdgePad"],["lx","ly","gx","gy"], ["dPedestals","Noise","NoiseRF0_Ratio","N_ClustersRF0_Ratio","N_DigitsRF0_Ratio"], {'sizing_mode': 'scale_width'}]
widgetLayoutDesc={
    "Select":widgetLayoutKine,
    "Histograms":[["nbinsX","nbinsY", "nbinsGX","nbinsGY","varX","varY","yAxisTransform"], {'sizing_mode': 'scale_width'}],
    "Legend": figureParameters['legend']['widgetLayout'],
    "Markers":["markerSize"]
}

#figureGlobalOption={"size":"size"}
figureGlobalOption={}
figureGlobalOption=figureParameters["legend"]["figureOptions"]
figureGlobalOption["size"]="markerSize"

## Histogram array
* histogram user defined - X,Y
* number of bins user defined

In [ ]:
histoArray=[    
    {
        "name": "histoXY",
        "variables": ["varX", "varY"],
        "nbins":["nbinsX", "nbinsY"], "axis":[0,1],"quantiles": [.1, .5, .9],"unbinned_projections":True
    },
    #
    {"name": "histoX", "variables": ["varX"], "nbins":"nbinsX", "range": None},
    #
    {"name": "histoY", "variables": ["varY"], "nbins":"nbinsY", "range": None, },
    {
        "name": "histoGXYYA",
        "variables": ["gx", "gy","varY"],
        "nbins":["nbinsGX", "nbinsGY","nbinsY"], "axis":[2],"quantiles": [.5],"unbinned_projections":True, "weights":"A_Side"
    },
    {
        "name": "histoGXYYC",
        "variables": ["gx", "gy","varY"],
        "nbins":["nbinsGX", "nbinsGY","nbinsY"], "axis":[2],"quantiles": [.5],"unbinned_projections":True, "weights":"C_Side"
    }
]

aliasArray=[]

## Create figure/application layout

In [ ]:
figureArray=[
    [["bin_center"],["bin_count"],{"source":"histoX","yAxisTitle":"N", "xAxisTitle":"varX", "errY": ["sqrt(bin_count)"]}],
    [["bin_center"],["bin_count"],{"source":"histoY","yAxisTitle":"N", "xAxisTitle":"varY", "errY": ["sqrt(bin_count)"]}],
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar": "log(bin_count+1)", "source":"histoXY"}],
    #
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar": "log(bin_count+1)", "source":"histoXY"}],
    [["bin_center_0"], ["mean","quantile_1"], { "source":"histoXY_1"}],
    [["bin_center_0"], ["std"], { "source":"histoXY_1"}],
    # global XY profile
     [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar": "quantile_0", "source":"histoGXYYA_2"}],
     [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar": "quantile_0", "source":"histoGXYYC_2"}],

    figureGlobalOption
]
figureLayoutDesc={
    "histo":[[0,1,2],{"plot_height":350}],
    "projection":[[3,4,5],{"plot_height":350}],
    "gxy":[[6,7],{"plot_height":350}],
}


In [ ]:
#%pdb
output_file("calibPad.html")
arrayCompression=arrayCompressionRelative10
#dfDraw=dfScan.sample(600000).sort_index()
dfDraw=dfScan
bokehDrawSA.fromArray(dfDraw, None, figureArray, widgetParams, layout=figureLayoutDesc, parameterArray=parameterArray, aliasArray=aliasArray,
                              widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", nPointRender=80000, histogramArray=histoArray,
                     arrayCompression=arrayCompression)